## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-21-04-14-14 +0000,nyt,Another Train Crash in Spain Kills 1 and Hurts...,https://www.nytimes.com/2026/01/20/world/europ...
1,2026-01-21-04-07-31 +0000,nyt,Air Force One Turns Back After Electrical Issue,https://www.nytimes.com/2026/01/20/us/politics...
2,2026-01-21-04-00-00 +0000,wsj,Russia Cheers the Growing NATO Rift Over Green...,https://www.wsj.com/world/russia/russia-cheers...
3,2026-01-21-03-59-52 +0000,bbc,Tourist helicopter goes missing near volcano i...,https://www.bbc.com/news/articles/c394pp3zkwlo...
4,2026-01-21-03-58-37 +0000,startribune,The latest: JD Vance to visit Minnesota this w...,https://www.startribune.com/ice-raids-minnesot...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2476/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
43,trump,99
17,greenland,44
250,new,19
33,court,16
126,world,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
382,2026-01-20-06-39-00 +0000,wsj,President Trump has ramped up pressure on Euro...,https://www.wsj.com/politics/policy/why-an-emb...,213
307,2026-01-20-13-48-20 +0000,nypost,Trump trolls European leaders with wild AI ima...,https://nypost.com/2026/01/20/us-news/trump-tr...,206
341,2026-01-20-10-36-28 +0000,bbc,"Greenland 'will stay Greenland', former Trump ...",https://www.bbc.com/news/articles/c98p6m0r53no...,205
56,2026-01-21-01-05-00 +0000,wsj,President Trump’s campaign for Greenland follo...,https://www.wsj.com/politics/policy/trumps-gre...,188
12,2026-01-21-03-26-41 +0000,nypost,Trump teases how far he’ll go to acquire Green...,https://nypost.com/2026/01/20/us-news/trump-te...,186


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
382,213,2026-01-20-06-39-00 +0000,wsj,President Trump has ramped up pressure on Euro...,https://www.wsj.com/politics/policy/why-an-emb...
319,70,2026-01-20-12-09-26 +0000,cbc,EU chief promises 'unflinching' response to ne...,https://www.cbc.ca/news/world/greenland-europe...
344,70,2026-01-20-10-30-00 +0000,wsj,Wednesday’s Supreme Court arguments over wheth...,https://www.wsj.com/economy/central-banking/je...
278,53,2026-01-20-15-58-06 +0000,nypost,Trump tells The Post why he posted Emmanuel Ma...,https://nypost.com/2026/01/20/us-news/trump-re...
19,51,2026-01-21-03-00-55 +0000,bbc,Driver killed in Spain train crash days after ...,https://www.bbc.com/news/articles/c1m78xl0gmpo...
51,47,2026-01-21-01-17-00 +0000,wsj,Escalating tensions over Greenland are superch...,https://www.wsj.com/economy/global/united-stat...
261,45,2026-01-20-17-00-00 +0000,wsj,The $0 Day: Small Minneapolis Businesses Ride ...,https://www.wsj.com/us-news/on-lake-street-in-...
43,44,2026-01-21-01-47-33 +0000,nypost,Minnesota teacher insults student’s intelligen...,https://nypost.com/2026/01/20/us-news/minnesot...
287,44,2026-01-20-15-15-07 +0000,latimes,Netflix amends Warner Bros. deal to all cash i...,https://www.latimes.com/entertainment-arts/bus...
285,44,2026-01-20-15-16-37 +0000,nypost,"UK considering its own Australia-style, teen s...",https://nypost.com/2026/01/20/world-news/uk-co...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
